In [1]:
import pandas as pd
import olefile
import zlib
import struct

# 한글 파일 전체를 불러오는 함수

In [2]:
def get_hwp_text(filename):
    #f = olefile.OleFileIO(filename)
    with olefile.OleFileIO(filename) as f:
        dirs = f.listdir()

        # HWP 파일 검증
        if ["FileHeader"] not in dirs or \
           ["\x05HwpSummaryInformation"] not in dirs:
            raise Exception("Not Valid HWP.")

        # 문서 포맷 압축 여부 확인
        header = f.openstream("FileHeader")
        header_data = header.read()
        is_compressed = (header_data[36] & 1) == 1

        # Body Sections 불러오기
        nums = []
        for d in dirs:
            if d[0] == "BodyText":
                nums.append(int(d[1][len("Section"):]))
        sections = ["BodyText/Section"+str(x) for x in sorted(nums)]

        # 전체 text 추출
        text = ""
        for section in sections:
            bodytext = f.openstream(section)
            data = bodytext.read()
            if is_compressed:
                unpacked_data = zlib.decompress(data, -15)
            else:
                unpacked_data = data
    
            # 각 Section 내 text 추출    
            section_text = ""
            i = 0
            size = len(unpacked_data)
            while i < size:
                header = struct.unpack_from("<I", unpacked_data, i)[0]
                rec_type = header & 0x3ff
                rec_len = (header >> 20) & 0xfff

                if rec_type in [67]:
                    rec_data = unpacked_data[i+4:i+4+rec_len]
                    section_text += rec_data.decode('utf-16')
                    section_text += "\n"

                i += 4 + rec_len

            text += section_text
            text += "\n"

    return text

In [8]:
schedule = pd.read_excel('일정표1.xlsx')
schedule

,회의록구분,대수,회의구분,위원회,회수,피감사기관,회의일자
0,서울특별시 시의회,10,행정사무감사,교육위원회,제303회[정례회](2021.11.01 ~ 2021.12.22),"서울시교육청 대변인, 감사관, 총무과, 기획조정실, 11개 교육지원청, 교육연구정보...",2021.11.02 화요일
1,서울특별시 시의회,10,행정사무감사,교육위원회,제303회[정례회](2021.11.01 ~ 2021.12.22),"서울시교육청 대변인, 감사관, 총무과, 기획조정실, 11개 교육지원청, 교육연구정보...",2021.11.03 수요일
2,서울특별시 시의회,10,행정사무감사,교육위원회,제303회[정례회](2021.11.01 ~ 2021.12.22),"과학전시관, 교육연수원, 유아교육진흥원",2021.11.04 목요일
3,서울특별시 시의회,10,행정사무감사,교육위원회,제303회[정례회](2021.11.01 ~ 2021.12.22),"평생진로교육국, 학생교육원, 학교보건진흥원, 학생체육관, 평생학습관, 도서관 (1)",2021.11.08 월요일
4,서울특별시 시의회,10,행정사무감사,교육위원회,제303회[정례회](2021.11.01 ~ 2021.12.22),"평생진로교육국, 학생교육원, 학교보건진흥원, 학생체육관, 평생학습관, 도서관 (2)",2021.11.09 화요일
5,서울특별시 시의회,10,행정사무감사,교육위원회,제303회[정례회](2021.11.01 ~ 2021.12.22),"교육행정국, 교육시설관리본부, 학교안전공제회 (1)",2021.11.10 수요일
6,서울특별시 시의회,10,행정사무감사,교육위원회,제303회[정례회](2021.11.01 ~ 2021.12.22),"교육행정국, 교육시설관리본부, 학교안전공제회 (2)",2021.11.11 목요일
7,서울특별시 시의회,10,행정사무감사,교육위원회,제303회[정례회](2021.11.01 ~ 2021.12.22),종합감사 및 총평,2021.11.12 금요일


In [11]:
date_list = ['1.11.2.속기록', '2.11.3.속기록', '3.11.4.속기록', '4.11.8.속기록', 
             '5.11.9.속기록', '6.11.10.속기록', '7.11.11.속기록', '8.11.12.속기록']


In [12]:
def createDirectory(directory): 
    """ 새로운 폴더를 생성하는 함수 """
    try: 
        if not os.path.exists(directory): 
            os.makedirs(directory) 
    except OSError: 
        print("Error: Failed to create the directory.")

In [13]:
import os

for d in range(len(date_list)):
    file_list = os.listdir(date_list[d])
    file_count = len(file_list)
    
    createDirectory(date_list[d] + 'csv version')
    for f in range(file_count):
        hwp = file_list[f]
        
        # 한글 파일 텍스트로 불러오기
        t = get_hwp_text(os.path.join(date_list[d], hwp))

        # 속기록 파일에 없는 컬럼들 기본 설정 (추후 폴더명에서 날짜 가져오는 등 수정 필요)
        proceedings_txt = schedule.loc[d]['회의록구분']
        daesu_txt = schedule.loc[d]['대수']
        conference_txt = schedule.loc[d]['회의구분']
        committee_txt = schedule.loc[d]['위원회'] 
        hoesu_txt = schedule.loc[d]['회수']
        auditee = schedule.loc[d]['피감사기관']
        date = schedule.loc[d]['회의일자']

        # 텍스트 데이터를 정제하여 리스트에 추가

        # '○' 을 기준으로 1차로 구분. 처음은 의미가 없는 문자들이라 제외.
        t_split = t.split('○')[1:]
        li = []
        i = 1
        for line in t_split:
            """ \r\n을 A로 치환한 뒤, A를 기준으로 텍스트를 나눈다.
            그리고 이를 발언자와 내용으로 구분한다. """
            temp = line.replace('\r\n', 'A')
            one = temp.split('A')
            person, speak = one[0], one[1:]    
    
            """ 1. XXX 위원 
                2. 위원
                해당 경우에서 person의 형식을 보고 기관구분을 분리하여 판단함.
            """
            person_li = person.split()
            if len(person_li) > 1:
                center = person_li[1]
            else:
                center = person_li[0]
            """ 회의록 형식에 맞추어 append.
            발언 내용의 경우 리스트를 ' '을 구분자로 하여 문자열을 합쳐주고, 처음과 마지막의 공백을 제거하여 추가하였다. """
            li.append([proceedings_txt, daesu_txt, conference_txt, committee_txt, hoesu_txt, auditee, date, person, i, ' '.join(speak).strip(), center])
            i += 1

        # 데이터프레임 생성

        df = pd.DataFrame(li, columns = ['회의록구분', '대수', '회의구분', '위원회', '회수', '피감사기관', '회의일자', '발언자', '발언순번', '발언내용', '기관구분'])
        df

        # csv 파일로 저장.

        df.to_csv(os.path.join(date_list[d] + "csv version", "%s.csv" % hwp[:-4]), index = False, encoding = 'euc-kr')
        print("%s -> %s 완료" %(date_list[d], hwp[:-4]))

1.11.2.속기록 -> 1 황인구 위원 완료
1.11.2.속기록 -> 10 문영민 위원 완료
1.11.2.속기록 -> 11전병주 위원 완료
1.11.2.속기록 -> 12 김용연 위원 완료
1.11.2.속기록 -> 13 이호대 위원 완료
1.11.2.속기록 -> 14 이동현 위원 완료
1.11.2.속기록 -> 15 양민규 위원 완료
1.11.2.속기록 -> 16 문영민 위원 완료
1.11.2.속기록 -> 17 김상진 위원 완료
1.11.2.속기록 -> 18 권순선 위원 완료
1.11.2.속기록 -> 19 황인구 위원 완료
1.11.2.속기록 -> 2 이석주 위원 완료
1.11.2.속기록 -> 20 양민규 위원 완료
1.11.2.속기록 -> 3 이동현 위원 완료
1.11.2.속기록 -> 4 이호대 위원 완료
1.11.2.속기록 -> 5 권순선 위원 완료
1.11.2.속기록 -> 6 위원장 완료
1.11.2.속기록 -> 7 김생환 위원 완료
1.11.2.속기록 -> 8 김수규 위원 완료
1.11.2.속기록 -> 9 양민규 위원 완료
2.11.3.속기록 -> 1 자료요구 완료
2.11.3.속기록 -> 10 김생환 위원 완료
2.11.3.속기록 -> 11 전병주 위원 완료
2.11.3.속기록 -> 12 이동현 위원 완료
2.11.3.속기록 -> 13이호대 위원 완료
2.11.3.속기록 -> 14 김용연 위원 완료
2.11.3.속기록 -> 15 문영민 위원 완료
2.11.3.속기록 -> 16 김생환 위원 완료
2.11.3.속기록 -> 17 전병주 위원 완료
2.11.3.속기록 -> 18 이동현 위원 완료
2.11.3.속기록 -> 19 이호대 위원 완료
2.11.3.속기록 -> 2 김상진 위원 완료
2.11.3.속기록 -> 20 감평 완료
2.11.3.속기록 -> 3 권순선 위원 완료
2.11.3.속기록 -> 4 황인구 위원 완료
2.11.3.속기록 -> 5 양민규 위원 완료
2.11.3.속기록 -> 6 이석주 위원 완료
2.11.3.속기록 -> 7 김용연 위원 완료
2